In [1]:
using Pkg;

In [2]:
FLAG = true;
if ! @isdefined(PDDL) && FLAG
    Pkg.add("PDDL")
    Pkg.add("PDDLViz")
    Pkg.add("GLMakie")
    Pkg.add("PlanningDomains")
    Pkg.add(url="https://github.com/JuliaPlanners/SymbolicPlanners.jl")
else
    println("Skipped reinstallation")
end;

    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
   Installed ValSplit ──────────── v0.1.1
   Installed ExprTools ─────────── v0.1.10
   Installed AutoHashEquals ────── v2.2.0
   Installed Julog ─────────────── v0.1.15
   Installed DocStringExtensions ─ v0.9.3
   Installed PDDL ──────────────── v0.2.18
   Installed ParserCombinator ──── v2.2.1
    Updating `/opt/julia/environments/v1.9/Project.toml`
  [2c8894f9] + PDDL v0.2.18
    Updating `/opt/julia/environments/v1.9/Manifest.toml`
  [15f4f7f2] + AutoHashEquals v2.2.0
  [ffbed154] + DocStringExtensions v0.9.3
  [e2ba6199] + ExprTools v0.1.10
  [5d8bcb5e] + Julog v0.1.15
  [2c8894f9] + PDDL v0.2.18
  [fae87a5f] + ParserCombinator v2.2.1
  [0625e100] + ValSplit v0.1.1
Precompiling project...
  ✓ AutoHashEquals
  ✓ ExprTools
  ✓ DocStringExtensions
  ✓ ValSplit
  ✓ Julog
  ✓ ParserCombinator
  ✓ PDDL
  7 dependencies successfully precompiled in 23 seconds. 61 already precompiled.
   Resolvin

In [1]:
using PDDL, PlanningDomains, SymbolicPlanners

In [185]:
compounds = initial.types
items = ["ingredient", "tool", "dish"]
items_instances = []
locations_instances = []
status_instances = []

# Iterate over the Set and extract the type and value
for compound in compounds
    # Extract type and value by splitting at '(' and ')'
    TYPE = compound.name
    INSTANCE = compound.args[1].name # Julia arrays are 1-indexed, so args[2] refers to the second element
    println(INSTANCE)

    if TYPE in items
        push!(items_instances, INSTANCE)
    elseif TYPE == "status"
        push!(status_instances, INSTANCE)
    elseif TYPE == "location"
        push!(locations_instances, INSTANCE)
    end
end

all_combinations = [] # All combinations of the three extracted lists adding ":" at the beginning of the string
println(items_instances)
# Generate all combinations
for comb in Iterators.product(items_instances, status_instances, locations_instances)
    println(comb)
    comb_str = join(comb, ":")
    push!(all_combinations, comb_str)

    # Check and update the initial values dictionary
    if !haskey(initial.values["item-count"], comb_str)
        initial.values["item-count"][comb_str] = 0
    end
end


dirty
sushi
fish
sushitool
sa
raw
pa
unprepared
prepared
rice
clean
seaweed
prepare
robot1
cooked
Any[]


In [206]:
compounds = initial.types
items = ["ingredient", "tool", "dish"]
items_instances = []
locations_instances = []
status_instances = []
# Iterate over the Set and extract the type and value
for compound in compounds
    # Extract type and value by splitting at '(' and ')'
    TYPE = compound.name
    INSTANCE = compound.args[1].symbol
    println(INSTANCE)
    if TYPE in items
        items_instances.append(value)
    if TYPE=="status"
        status_instances.append(value)
    if TYPE=="location"
        locations_instances.append(value)
end
all_combination = [] #All combinatiuons of the three stracted lists adding ":" at the beggining of the string
for comb in all_combinations
    if not comb in initial.values["item-count"]:
        initial.values["item-count"][comb] = 0
        
end

LoadError: syntax: line break in ":" expression

In [205]:
initial = initstate(a_domain, a_problem)
data = initial.values
for (key, inner_dict) in data
    println("Key: ", key)
    println("Items:")
    for (inner_key_tuple, value) in inner_dict
        println("  - Items: ", inner_key_tuple, typeof(inner_key_tuple), " => Count: ", value)
    end
    println()  # Adding a newline for better readability
end

Key: ingredient-count
Items:
  - Items: (:fish, :sa)Tuple{Symbol, Symbol} => Count: 1
  - Items: (:chocolat, :sa)Tuple{Symbol, Symbol} => Count: 1
  - Items: (:rice, :sa)Tuple{Symbol, Symbol} => Count: 1
  - Items: (:seaweed, :sa)Tuple{Symbol, Symbol} => Count: 1
  - Items: (:flour, :sa)Tuple{Symbol, Symbol} => Count: 1



In [38]:
a_domain = load_domain( "domain copy.pddl" )
a_problem = load_problem( "problem copy.pddl" )
initial = initstate(a_domain, a_problem)
the_goals = MinStepsGoal( a_problem )
a_planner = AStarPlanner(HAddR())
solved = a_planner( a_domain, initial, the_goals)

PathSearchSolution{...}
  status: :success
  plan: 47-element Vector{Term}
    (move robot1 pa sa)
    (pick-up-ingredient robot1 rice sa)
    (move robot1 sa pa)
    (move robot1 pa cla)
    (move robot1 cla ka)
    (drop robot1 rice ka)
    (cook robot1 rice ka)
    (pick-up-items robot1 rice ka)
    (move robot1 ka cla)
    (move robot1 cla pa)
    ⋮
    (move robot1 cla pa)
    (drop robot1 knife pa)
    (move robot1 pa sa)
    (pick-up-ingredient robot1 seaweed sa)
    (move robot1 sa pa)
    (drop robot1 seaweed pa)
    (move robot1 pa sa)
    (pick-up-ingredient robot1 fish sa)
    (move robot1 sa pa)
    (drop robot1 fish pa)
    (prepare-dish robot1 sushi pa)
  trajectory: 48-element Vector{GenericState}
  expanded: 379915
  search_tree: Dict{UInt64, SymbolicPlanners.PathNode{GenericState}} with 375610 entries
  search_frontier: DataStructures.PriorityQueue{UInt64, Tuple{Float32, Float32, Int64}, Base.Order.ForwardOrdering} with 4145 entries
  search_order: 379915-element Vect

In [39]:
# Open a file for writing
open("tree.txt", "w") do file
    for node in solved.search_order
        # Write each line to the file
        println(file, " ", solved.search_tree[node])
        println(file, "\n\n")
    end
end

In [194]:
solved.search_tree

Dict{UInt64, SymbolicPlanners.PathNode{GenericState}} with 56 entries:
  0x501a66e116051511 => PathNode{GenericState}(0x501a66e116051511, GenericState…
  0xa25cbbb159cd2467 => PathNode{GenericState}(0xa25cbbb159cd2467, GenericState…
  0x7457f4eeacaf7118 => PathNode{GenericState}(0x7457f4eeacaf7118, GenericState…
  0x1ba3443d742cb0c5 => PathNode{GenericState}(0x1ba3443d742cb0c5, GenericState…
  0x28366157455dd594 => PathNode{GenericState}(0x28366157455dd594, GenericState…
  0xd0141b3a76a4dfea => PathNode{GenericState}(0xd0141b3a76a4dfea, GenericState…
  0xd14c0412a5e9e9ff => PathNode{GenericState}(0xd14c0412a5e9e9ff, GenericState…
  0x910d7cac9b32cc84 => PathNode{GenericState}(0x910d7cac9b32cc84, GenericState…
  0xfd3b72c5a344e2fd => PathNode{GenericState}(0xfd3b72c5a344e2fd, GenericState…
  0x758d42a875e5beeb => PathNode{GenericState}(0x758d42a875e5beeb, GenericState…
  0x77c13058143255c6 => PathNode{GenericState}(0x77c13058143255c6, GenericState…
  0x7acb66c3c8b38b66 => PathNode{Gener

In [237]:
for node in solved.search_order
    println(" ", solved.search_tree[node])
    println("\n\n")
end;

 SymbolicPlanners.PathNode{GenericState}(0x8c0ace08fbb87ec3, GenericState(Set(Compound[status(dirty), status(cooked), dish(sushi), ingredient(fish), process(cooking), status(raw), location(pa), ingredient(flour), process(cleaning), ingredient(chocolate), process(preparing), ingredient(rice), status(clean), ingredient(seaweed), storage-location(sa), location(cla), tool(knife), dish(pollofre), location(ka), robot(robot1)]), Set(Term[done-at(cleaning, knife, cla), connected(sa, pa), cause-status(preparing, pollofre, knife, dirty), item-at(knife, clean, sa), item-at(fish, raw, sa), requires-item(preparing, pollofre, chocolate, cooked), applicable(raw, flour), requires-item(preparing, sushi, knife, clean), applicable(cooked, rice), done-at(preparing, sushi, pa), connected(pa, cla), dish-at(pollofre, pa), robot-at(robot1, pa), applicable(clean, knife), requires-item(preparing, sushi, rice, raw), item-at(rice, raw, sa), item-at(flour, raw, sa), requires-item(preparing, sushi, seaweed, raw), a

Excessive output truncated after 524288 bytes.

(pollofre, pa), robot-at(robot1, pa), requires-item(preparing, sushi, fish, raw), applicable(clean, knife), requires-item(preparing, sushi, rice, raw), item-at(rice, raw, sa), requires-item(preparing, pollofre, flour, raw), cause-status(preparing, sushi, knife, dirty), item-at(seaweed, raw, sa), applicable(cooked, fish), item-at(flour, raw, sa), requires-item(preparing, sushi, seaweed, raw), applicable(dirty, knife), done-at(cooking, chocolate, ka), dish-at(sushi, pa), priority-over(pollofre, sushi), requires-item(preparing, pollofre, knife, clean), applicable(raw, chocolate), connected(ka, cla), applicable(raw, seaweed), applicable(cooked, seaweed), item-at(chocolate, raw, sa), cause-status(cleaning, knife, knife, clean), done-at(preparing, pollofre, pa), connected(cla, ka), cause-status(cooking, chocolate, chocolate, cooked)]), Dict{Symbol, Any}(Symbol("ingredient-count") => Dict{Any, Any}((:fish, :sa) => 0, (:chocolate, :sa) => 0, (:rice, :sa) => 1, (:seaweed, :sa) => 0, (:flour, :s

In [195]:
for (index, node) in enumerate(solved.search_order)
    current_node = solved.search_tree[node]
    previous_node_ref = current_node.previous_ref
    action_taken = previous_node_ref ? previous_node_ref.action : "Root Node (No Action)"
    
    println("Node $(index):")
    println("    ID: ", node)
    println("    Previous Node: ", previous_node_ref ? previous_node_ref.id : "None")
    println("    Action Taken: ", action_taken)
    println("    State: ", current_node.state)
    println("\n")
end


LoadError: type PathNode has no field previous_ref

In [ ]:
solved.search_tree[0x6e2e8f67bb9bd060]

In [125]:
println("Whole plan: (", length(solved), ") steps")
for s in collect(solved)
    println(" ", s)
end;

Whole plan: (12) steps
 move(robot1, pa, sa)
 pick-up(robot1, seaweed, sa)
 move(robot1, sa, pa)
 use-ingredient(robot1, seaweed, pa)
 move(robot1, pa, sa)
 pick-up(robot1, rice, sa)
 move(robot1, sa, pa)
 use-ingredient(robot1, rice, pa)
 move(robot1, pa, sa)
 pick-up(robot1, fish, sa)
 move(robot1, sa, pa)
 use-ingredient(robot1, fish, pa)
